In [1]:
# | default_exp train.nbeats

In [2]:
# | export
from datasetsforecast.m3 import M3

df = M3().load("../data", group="Monthly")[0]
df.sort_values(["unique_id", "ds"], inplace=True)
df

/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packages/datasetsforecast/m3.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(class_group.freq)


,unique_id,ds,y
0,M1,1990-01-31,2640.0
1,M1,1990-02-28,2640.0
2,M1,1990-03-31,2160.0
3,M1,1990-04-30,4200.0
4,M1,1990-05-31,3360.0
...,...,...,...
167557,M999,1993-10-31,5225.9
167558,M999,1993-11-30,5236.3
167559,M999,1993-12-31,5186.6
167560,M999,1994-01-31,5143.4


In [3]:
df.unique_id.nunique()

1428

In [4]:
df.groupby("unique_id").apply(len).describe()

/tmp/ipykernel_167134/840638693.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("unique_id").apply(len).describe()


count    1428.000000
mean      117.340336
std        28.508554
min        66.000000
25%        96.000000
50%       133.000000
75%       134.000000
max       144.000000
dtype: float64

In [5]:
# | export
from ts.preprocess.dataloader import UnivariateTSDataModule

In [6]:
# | export
input_size = 60
horizon = 12
batch_size = 512
num_workers = 24
step_size = 3

ds = UnivariateTSDataModule(
    df=df,
    input_size=input_size,
    horizon=horizon,
    batch_size=batch_size,
    num_workers=num_workers,
    train_split=0.7,
    val_split=0.15,
    normalize=True,
    scaler_type="minmax",
    split_type="vertical",
    step_size=step_size,
)

In [7]:
# | export
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from ts.model.nbeats import NBeatsG

In [8]:
# | export
from pytorch_lightning.profilers import AdvancedProfiler

profiler = AdvancedProfiler(dirpath=".", filename="memory_profile.txt")

# Example trainer setup (without full NBeatsG for brevity)
trainer = pl.Trainer(
    max_epochs=200,  # Short run for testing
    accelerator="auto",
    logger=TensorBoardLogger("M4_logs", name="nbeatsg_m3_monthly"),
    callbacks=[EarlyStopping("val_loss", patience=20, verbose=False)],
    # profiler=profiler
)

model = NBeatsG(input_size=input_size, horizon=horizon)
trainer.fit(model, ds)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:ts.preprocess.dataloader:Train windows: 15826, Val windows: 3376, Test windows: 3489
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type                                 | Params | Mode 
-------------------------------------------------------------------------


Sanity Checking: |                                                                                            …

/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (31) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Filename: /home/pranav-pc/projects/ts/ts/model/nbeats.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    94   1252.9 MiB   1252.9 MiB           1       @profile
    95                                             def training_step(self, batch, batch_idx):
    96   1252.9 MiB      0.0 MiB           1           x, y = batch
    97   1253.1 MiB      0.2 MiB           1           y_hat = self(x)
    98   1253.1 MiB      0.0 MiB           1           loss = self.loss_fn(y_hat, y)
    99   1253.1 MiB      0.0 MiB           1           self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
   100   1253.1 MiB      0.0 MiB           1           return loss


Filename: /home/pranav-pc/projects/ts/ts/model/nbeats.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    94   1339.4 MiB   1339.4 MiB           1       @profile
    95                                             def training_step(self, batch, batch_idx):
    96   1339.4 MiB      0.0 Mi

...

Validation: |                                                                                                 …

Filename: /home/pranav-pc/projects/ts/ts/model/nbeats.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    94   1356.7 MiB   1356.7 MiB           1       @profile
    95                                             def training_step(self, batch, batch_idx):
    96   1356.7 MiB      0.0 MiB           1           x, y = batch
    97   1356.7 MiB      0.0 MiB           1           y_hat = self(x)
    98   1356.7 MiB      0.0 MiB           1           loss = self.loss_fn(y_hat, y)
    99   1356.7 MiB      0.0 MiB           1           self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
   100   1356.7 MiB      0.0 MiB           1           return loss


Filename: /home/pranav-pc/projects/ts/ts/model/nbeats.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    94   1356.7 MiB   1356.7 MiB           1       @profile
    95                                             def training_step(self, batch, batch_idx):
    96   1356.7 MiB      0.0 Mi

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


Filename: /home/pranav-pc/projects/ts/ts/model/nbeats.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    94   1356.7 MiB   1356.7 MiB           1       @profile
    95                                             def training_step(self, batch, batch_idx):
    96   1356.7 MiB      0.0 MiB           1           x, y = batch
    97   1356.7 MiB      0.0 MiB           1           y_hat = self(x)
    98   1356.7 MiB      0.0 MiB           1           loss = self.loss_fn(y_hat, y)
    99   1356.7 MiB      0.0 MiB           1           self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
   100   1356.7 MiB      0.0 MiB           1           return loss


Filename: /home/pranav-pc/projects/ts/ts/model/nbeats.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    94   1356.7 MiB   1356.7 MiB           1       @profile
    95                                             def training_step(self, batch, batch_idx):
    96   1356.7 MiB      0.0 Mi

NameError: name 'exit' is not defined

In [ ]:
# | export
trainer.test(model, ds);

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
from plotly.subplots import make_subplots


def forecast_and_plot_grid(trainer, model, data_module, num_series=6):
    # Ensure model is in evaluation mode
    model.eval()

    # Get all unique_ids from the DataFrame
    unique_ids = data_module.df["unique_id"].unique()
    # Randomly select 6 unique_ids (or fewer if less available)
    selected_ids = np.random.choice(
        unique_ids, size=min(num_series, len(unique_ids)), replace=False
    )

    # Prepare test data for selected series
    test_data = data_module.df[data_module.df["unique_id"].isin(selected_ids)]
    grouped = test_data.groupby("unique_id")

    # Create figure with 3x2 grid
    fig = make_subplots(
        rows=3,
        cols=2,
        subplot_titles=[f"Series: {uid}" for uid in selected_ids],
        vertical_spacing=0.15,
        horizontal_spacing=0.1,
    )

    # Perform forecasting and plotting
    for i, unique_id in enumerate(selected_ids):
        series_df = grouped.get_group(unique_id)
        series = series_df["y"].values
        series_len = len(series)

        if series_len < data_module.input_size + data_module.horizon:
            print(f"Skipping {unique_id}: too short for forecasting")
            continue

        # Normalize series if data_module uses normalization
        if data_module.normalize:
            scaler = data_module.scalers.get(unique_id)
            if scaler:
                series_normalized = scaler.transform(series.reshape(-1, 1)).flatten()
            else:
                series_normalized = series
        else:
            series_normalized = series

        # Generate last input window for forecasting
        last_input = series_normalized[
            -data_module.input_size - data_module.horizon : -data_module.horizon
        ]
        x = torch.tensor(last_input, dtype=torch.float32).unsqueeze(0).to(model.device)

        # Forecast
        with torch.no_grad():
            y_hat = model(x).cpu().numpy().flatten()

        # Inverse transform predictions and actual values
        if data_module.normalize and scaler:
            y_hat_denorm = scaler.inverse_transform(y_hat.reshape(-1, 1)).flatten()
            series_denorm = series  # Already in original scale
        else:
            y_hat_denorm = y_hat
            series_denorm = series

        # Time indices for the entire series and forecast
        full_time_indices = series_df["ds"].values  # Full series dates
        forecast_time_indices = full_time_indices[-data_module.horizon :]  # Last horizon dates

        # Determine row and column (0-based indexing, converting to 1-based for Plotly)
        row = (i // 2) + 1
        col = (i % 2) + 1

        # Plot the entire actual series
        fig.add_trace(
            go.Scatter(
                x=full_time_indices,
                y=series_denorm,
                mode="lines+markers",
                line=dict(color="blue"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

        # Plot predicted values for the last horizon
        fig.add_trace(
            go.Scatter(
                x=forecast_time_indices,
                y=y_hat_denorm,
                mode="lines+markers",
                line=dict(color="red", dash="dash"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

    # Update layout
    fig.update_layout(
        height=900,
        width=800,  # Fixed size for 3x2 grid
        title_text="N-BEATS-G Forecasting: Full Series with Last Horizon Predicted (3x2 Grid)",
        showlegend=False,  # Remove legend entirely
    )
    fig.update_yaxes(title_text="Value")
    fig.update_xaxes(title_text="Date")

    # Show plot
    fig.show()


# Example usage (assuming trainer, model, and data_module are defined)
# Replace 'ds' with 'data_module' if that’s your variable name
forecast_and_plot_grid(trainer, model, ds, num_series=6)

In [ ]:
# | export
trainer.save_checkpoint("M3-MONTHLY-profile.ckpt")